In [ ]:
!pip install kaggle
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od
 
od.download("https://www.kaggle.com/datasets/stackoverflow/stacksample/")


In [ ]:
import pandas as pd
from io import StringIO

In [ ]:
# Importing the Questions.csv file

questions_df = pd.read_csv('stacksample/Questions.csv', encoding = "ISO-8859-1")
questions_df.head()

In [ ]:
# Importing the Answers.csv file

answers_df = pd.read_csv('stacksample/Answers.csv', encoding = "ISO-8859-1")
answers_df.head()

In [ ]:
# Importing the Tags.csv file

tags_df = pd.read_csv('stacksample/Tags.csv', encoding = "ISO-8859-1")
tags_df.head()

## Data processing

In [ ]:
# Adding number of answers to each question
cout_of_answers = answers_df.groupby('ParentId').count() # Counting the number of answers for each question
proccessed_df = pd.merge(questions_df, cout_of_answers, left_on='Id', right_on='ParentId') # Merging the questions and answers dataframes

proccessed_df.head()

In [ ]:
# Adding tags to each question
tags_list_df = tags_df.groupby('Id')['Tag'].agg(list).reset_index(name='TagsList')  # Grouping the tags by question id
tags_list_df.head()
proccessed_df = pd.merge(proccessed_df, tags_list_df, left_on='Id', right_on='Id') # Merging the questions and tags dataframes
# ^ TODO: Fix this merge, it doesn't work
# # Adding number of tags to each question
# proccessed_df['CountOfTags'] = proccessed_df['Tag'].apply(len)

# proccessed_df.head()